https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.boxcox.html

При решении задач регрессии используются именно регрессоры. Из хороших классических алгоритмов - SVM

В этой тетради мы будем исследовать наш df  на предмет пропусков, нулевых значений выбросов и занчений ,которые противоречат физике

In [ ]:
# Импортируем в проект требуемые библиотеки

import pandas as pd # библиотека для обработки и анализа данных
import numpy as np
import seaborn as sns # библиотека для создания статистических графиков
import random #  генераторатор случайных чисел и данных
import os # библиотека функций для работы с операционной системой.

from sklearn. preprocessing import OneHotEncoder

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
%matplotlib inline
sns.set_style('darkgrid')


import ydata_profiling # pandas_profiling!!!! отменяется c 1-го апреля 2023 г

import warnings
warnings.filterwarnings("ignore")
matplotlib.style.use('seaborn-pastel')


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import scipy

from scipy import stats # для метода обнаружения Q и IQR,  а также z-корреляции
from scipy.stats import mstats

np.random.seed(45) # чтобы выборки данных всегда были одни и теже берем любую цифру . мне нравится 45!


# Этап 3.3.

# Проверка того, есть ли в структуре данных какие-либо пропущенные значения

# Прочитаем наш df - 'data_composite_inspect.csv' из тетради Diploma1_data

In [ ]:
data_composite_inspect = pd.read_csv(r'C:\Users\grain\Work_folder\Diplom_MGTU\Diploma1_data\data_composite_inspect.csv')
data_composite_inspect.head()

In [ ]:
data_composite_inspect.describe()

In [ ]:
data_composite_inspect.info()

В нашем файле 1023 строки и 13 признаков-столбцов. Все значения у нас числовые, тип данных float

In [ ]:
Проверим отсутствие: пропущенных и нулевых значений, а также дубликатов строк

In [ ]:
# метод .isnull() выдает логический массив, где пропуски обозначены как True.
# Функция .isnull() используется для проверки того, есть ли в структуре данных какие-либо пропущенные значения.
# метод .sum() по умолчанию суммирует эти True или единицы по столбцам (axis = 0)
# Можно использовать функцию .isnull() вместе с .sum(), чтобы увидеть количество пропущенных значений в каждом столбце.

data_composite_inspect.isnull().sum()

Пропущенных значений нет!

Проверка того, есть ли в структуре данных какие-либо отсутствующих значений (NaN)

In [ ]:
# Функция .isna() в Pandas используется для обнаружения отсутствующих значений (NaN), 
# значения NaN в структуре данных сопоставляются с True, а значения, отличные от NaN, сопоставляются с False.
# Можно использовать функцию isna вместе с sum, чтобы увидеть количество пропущенных значений в каждом столбце.

data_composite_inspect.isna().sum() 

In [ ]:
# Еще один метод определения пропущенных значений - Тепловая карта пропущенных значений sns.heatmap

cols = data_composite_inspect.columns[:13] # первые 13 колонок
# определяем цвета 
# желтый - пропущенные данные, синий - не пропущенные
colours = ['#000099', '#ffff00'] 
sns.heatmap(data_composite_inspect[cols].isnull(), 
            cmap=sns.color_palette(colours)
           )

Мы определили, что в нашем DataFrame data_composite_inspect нет  пропущенных значений и отсутствующих значений (NaN)

Проверим наш DataFrame на наличие дубликатов  строк 

In [ ]:
# Найдем ,если они есть количество дублируеых строк
count_dupl = len(data_composite_inspect)-len(data_composite_inspect.drop_duplicates())

In [ ]:
print(count_dupl)

В нашем dataFrame НЕТ дубликатов

In [ ]:
# подсчет уникальных значеий по каждому признаку
df1.apply(lambda x: x.nunique())

In [ ]:
# проверяем наличие дубликатов в данных
df1.duplicated().sum()

In [ ]:
!!!!!!!
df1.describe().T

# Продолжаем исследование нашего df - data_composite_inspect

In [ ]:
# применим метод .describe() к количественным признакам
# округлим значения до 2-х знаков после запятой - round(3)
data_composite_inspect.describe(percentiles = [0.25, 0.5, 0.75], include='all').round(2)

In [ ]:
# Полный анализ нашего df  можно осуществить с помощью модуля df.profile_report()
data_composite_inspect.profile_report()

# Как выяснилось ,в нашем df есть один категориальный параметр - Угол нашивки- pattern_angle
# Переведем этот параметр в числовой 
Используем модуль OneHotEncoding 
Использовать LabelEncoder  !!!!!!


In [ ]:
# создание экземпляра one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

In [ ]:
# выполните однократное кодирование в столбце "pattern_angle" 
encoder_df = pd.DataFrame(encoder.fit_transform(data_composite_inspect[['pattern_angle']]).toarray())

In [ ]:
encoder_df.info()

In [ ]:
# объедините столбцы с односторонним кодированием обратно с исходным фреймом данных
data_composite_inspect = data_composite_inspect.join (encoder_df)

In [ ]:
data_composite_inspect.info()

In [ ]:
# удаляем исходный столбец 'pattern_angle' column
data_composite_inspect.drop('pattern_angle', axis= 1 , inplace= True )


In [ ]:
data_composite_inspect.info()

In [ ]:
data_composite_inspect.columns

In [ ]:
# Переименуем нове столбцы '0' на 'pattern_0_angle', а '90' на 'pattern_90_angle'

In [ ]:
#rename columns
data_composite_inspect.columns = ['step_strip', 'density_strip',
                    'ratio_filler_matrix', 'density',
                    'elasticity_module', 'number_hardeners',
                    'content_epoxy_groups','flash_temperature',
                    'surface_density', 'elasticity_module_stretching',
                    'strapery_strength', 'resin_consumption',
                    'pattern_0_angle', 'pattern_90_angle']

In [ ]:
data_composite_inspect.head(5)

In [ ]:
# Еще раз проведем полный анализ нашего df  с помощью модуля df.profile_report()
data_composite_inspect.profile_report()

# Первичный Графический анализ

In [ ]:
# Построение графиков
# Взято из https://pyprog.pro/sns/sns_6_visualization_of_dist.html

# Функция построения основных графиков по парметрам df:  sns.histplot и sns.boxplot, sns.kdeplot, sns.scatterplot и sns.displot
# Параметры, передаваемые в функцию:
# df- наш dataframe
# plt_type - тип графика ,который выводится (histplot, boxplot, displot, kdeplot, catterplot)
# n_col - количество столбцов графиков сколько будем строить в строке
# figsize_a - размер по ширине
# figsize_b - размер по высоте

def drawing_graphs(df, n_col, figsize_a, figsize_b, plt_type): # figsize_a, figsize_b - было 12,2
    
    fig, axes = plt.subplots(1, n_col, figsize=(figsize_a, figsize_b)) 
    i = 0              
    n_colum  = df.shape[1]  # количество столбцов в df / если 1, то количество столбцов (13), если 0, то строк 1023 !
    name_type_plt = plt_type
    
    for column in df.columns:
        if  name_type_plt == 'histplot':
            sns.histplot(df[column],ax=axes[i],
                         bins = 30, 
                         kde = True, # оценка плотности ядра (kernel density estimation или сокращенно kde)
                         fill = True
                         )    
        elif   name_type_plt == 'boxplot':       
            sns.boxplot(data=df, ax=axes[i],
                        x=column,
                        )
        elif  name_type_plt == 'kdeplot':
             sns.kdeplot(df[column], ax=axes[i], 
                         bw_adjust= 1
                        ) 
        elif  name_type_plt == 'displot':
             sns.displot(data =df, ax=axes[i],
                         x = column, 
                         kind = 'kde',
                         fill = True
                         )

        elif  name_type_plt == 'scatterplot':
            sns.scatterplot(df[column], ax=axes[i],
                            alpha=1,
                            color='b'
                            )
        i = i + 1
        if i > (n_col - 1) or column == df.columns[n_colum - 1]:
            plt.show()
            fig, axes = plt.subplots(1, n_col, figsize=(figsize_a, figsize_b))
            i = 0
    return

In [ ]:
drawing_graphs(data_composite_inspect, 2, 12, 2, 'histplot')

In [ ]:
drawing_graphs(data_composite_inspect, 3, 10, 4, 'scatterplot')

# Внимание !!!! первые 40 строк DF  имеют явно искаженный характер и являются искусственно привнесенными данными

In [ ]:
drawing_graphs(data_composite_inspect.head(50), 2, 8, 2, 'scatterplot')

# Все три признака, представленные в датасете df_nup по первым 20 значениям можно расценивать как дискретные, что вполне оправдано для представленных категорий, однако в диапазоне от 20 и до 1022 данные в колонках "Плотность нашивки" и "Шаг нашивки" представлены непрерывными значениями.
Возможно, данные в диапазоне (20; 1022) были подвергнуты преобразованию.

# Разделим наш датафрейм на три и произведем обучение моделей на двух датафреймах и сопоставим результаты: Датафрейм № 1 - объединенный по индексам датасет без разделения; Датафрейм № 2 - объединенный по индексам датасет за исключением!!!!!!!  первых 39 строк с дискретными значениями; 
Датафрейм № 3 - экспериментальный, исключающий все дискретные значения;

Рссмотрbv DF 2 хотя он и имеет в виду очень маленькую выборку данных.

In [ ]:
df3 = df.drop(labels=range(0, 39), axis=0)

In [ ]:
# __________________________________________

In [ ]:
sns.scatterplot(data_composite_inspect['density_strip'], alpha=1)

Взято из GitHub
# Построим гистограммы распределения
a = 5 # количество строк
b = 5 # количество столцбцов
c = 1 # инициализация plot counter
plt.figure(figsize = (35,35))
plt.suptitle('Гистограммы переменных', fontsize = 30)
for col in df.columns:
    plt.subplot(a, b, c)
    sns.histplot(data = df[col], kde=True, color = "darkblue")
    plt.ylabel(None)
    plt.title(col, size = 20)
    c += 1 

In [ ]:
# Графики зависимости параметров друг от друга с плотностью распределения
# Нужно ли?
sns.pairplot(data_composite_inspect.head(100), diag_kind='kde') #, hue='pattern_angle') df_elements = df.sample(n=7)

In [ ]:
# Проведем анализ данных в нашем df: data_composite_inspect на предмет 'подозрительных значений' и выбросов

При анализе df.discribe  и df.profile_report() выяснилось ,что в столбцах 'Шаг нашивки' и 'Плотность нашивки'
min значения равны 0.0 ,что не может быть физически.

А также значения MIN в столбцах:'ratio_filler_matrix'  = 0.389 'этот параметр  не может быть меньше 1,
'elasticity_module'= 2.437 это значение на 2 порядка меньше среднего- явная ошибка заполнения,
surface_density = 0.604 то значение на 3 порядка меньше среднего- явная ошибка заполнения.

Эти МИН значения несоотвествуют физически
м параметрм нашего проекта относительно показателей для композитных материлов.
И являютя выбросми 'слева'.

In [ ]:
def column_value_counts(df):
    for column in df.columns:
        print('столбец : ', column)
        print('Интервал значений', '  ', 'Количество в интервале')
        print(df[column].value_counts(bins=50))
        print('-----------------------------')
    return

In [ ]:
column_value_counts(data_composite_inspect)

# Мы можем определить интервалы значений, которые можно отбросить  в первом приближении по МИН значению,
как противоречищий физическому смыслу: 
1-й интервал (0.527, 66.074]    строк 16
 2-й интервал :
'''
1529.716, 1593.353]     5
(1593.353, 1656.989]     3
(1784.263, 1847.899]     1
(1847.899, 1911.536]     1
(1656.989, 1720.626]     0
(1720.626, 1784.263]     0
'''
и так получим значения крайних -левых значений для DataFrame для каждого столбца в df_min_max!!!

In [ ]:
# Создадим DataFrame  df_min_max который содержит мин и мах значения по каждому столбу в качестве критерия отбрасывания строк 
# min - индекс сроки 0 и max- индекс строки 1 получены оценочным путем при исследовании гистограмм по каждому параметру-столбцу
# Для столбца 'pattern_angle'диапазон будет :[0.0, 90.0],

df_min_max = pd.DataFrame({
                     
                      'step_strip':[1.5, 13.5],
                  'density_strip' :[29.0, 85.6],
             'ratio_filler_matrix':[0.5, 6.0],
                        'density' :[1780.0, 2159.0],
              'elasticity_module' :[50.0, 1560.0],
                'number_hardeners':[30.0, 190.0],     
            'content_epoxy_groups':[16.0, 28.0],
               'flash_temperature':[180.0, 390.0],
                'surface_density' :[20.0, 1200.0],
    'elasticity_module_stretching':[66.0, 81.0],
              'strapery_strength' :[1326.0, 3600.0],
               'resin_consumption':[80.0, 350.0],
                 'pattern_0_angle':[0.0, 0.0],
                'pattern_90_angle':[1.0, 1.0]
                          })

df_min_max

In [ ]:
# Функция расчета СРЕДНЕГО значения для "подозрительного элемента" в столбце с разбросом 
# как ВЫШЕ так и НИЖЕ от "подозрительного элемента"
# Передаваемые в функцию параметры:
# df[column] - столбец значений в котором сейчас идет проверка
# n_row -      номер строки "подозрительного элемента"
# n_spread -   задаваемый разброс обычно я задаю 5.

def mean_round_point(df, column, n_row, n_spread):
    mean_n_spread = 0 # возвращаемое значение среднего, полученного из суммы ближайших (n_spread * 2) значений столбца
    
    higth_value_sum = 0 # Сумма значений, где текущее значение номера строки Меньше (n_spread) 
    low_value_sum = 0   # Сумма значений,  где текущее значение номера строки Больше (n_spread)
    
# Для значений столбца, где текущее значение номера строки Меньше (n_spread) - заданного интервала для определения СРЕДНЕГО значения
    for n in range(n_spread):
        if (n_row == 0) or (n_row - n < 0):
            break
        else:
            higth_value_sum = higth_value_sum + df.loc[n_row - n][column]
            n = n + 1    
# Для значений столбца, где текущее значение номера строки Больше (n_spread) - заданного интервала для определения СРЕДНЕГО значения
    for k in range(n_spread):   
        if n_row + k >= len(df[column]):  
            break
        else:
            low_value_sum = low_value_sum + df.loc[n_row + k][column]    
            k = k + 1
# Рассчитываем суммы n_spread элементов выше (left_value_sum)  и ниже (rigth_value_sum) 'подозрительного элемента' строки n_row
    mean_n_spread =  (higth_value_sum + low_value_sum) / (n + k)
    print('новое значение :', mean_n_spread)
    return mean_n_spread

In [ ]:
def clean_df_left(df, df_min_max):
    sum_n_quantity = 0 # Общее количество подозрительных строк в df

    for column in df.columns:
        if column == 'pattern_0_angle' or column == 'pattern_90_angle':
            break
        n_col = column
        print('подозрительный столбец : ', n_col)
        print('экспертное минимально-допустимое значение :', df_min_max.loc[0, n_col])
    
        n_quantity = 0 # Количество подозрительных строк в столбце
       
        for i in range(len(df)):
            if df.loc[i, n_col] < df_min_max.loc[0, n_col]:
                n_quantity =  n_quantity + 1
                print('строка N:', i)
                print('старое значение :', df.loc[i, n_col])      
            # Функция 'mean_round_point' расчета СРЕДНЕГО значения для "подозрительного элемента" в столбце с разбросом '5'
                df.at[i, n_col] = mean_round_point(df, n_col, i, 5) 
        print('Количество подозрительных строк в столбце = ', n_quantity)
        print('---------------------------------')
        sum_n_quantity = sum_n_quantity + n_quantity
    print('Общее количество подозрительных строк в df = ', sum_n_quantity)
    return(df)

In [ ]:
clean_df_left(data_composite_inspect, df_min_max)

всего замена подозрительных значений слева произведена в 106 строках

Проведем очистку нашег df (в котором 1023 строки) от выбросов

Способ удаления выбросов IQR.
Записшем очищеный массив в df - data_clean

In [ ]:
# https://www.codecamp.ru/blog/remove-outliers-python/
# find Q1, Q3, and interquartile range for each column
# создаем новый df: data_clean

def outliers_delit(df):
    Q1 = df.quantile(q=.25)
    Q3 = df.quantile(q=.75) 
    IQR = df.apply(stats.iqr) # from scipy import stats

#only keep rows in dataframe that have values within 1.5\*IQR of Q1 and Q3

    data_clean = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

#find how many rows are left in the dataframe 

    data_clean.shape
    print('Количество строк до удаления: ', len(df))
    print('Количество строк после удаления: ', len(data_clean))
    print('Удалено строк :', (len(df) - len(data_clean)))
    
    return(data_clean)

In [ ]:
data_clean = outliers_delit(data_composite_inspect)

#### У нас было 1023 строки  до применения метода IQR. Теперь мы получили после очистки 936 строк в df -  data_clean.
Удлили 61 строк

второй способ z-с  посмотрим сколько строк уберет этот метод из  массива data_clean

In [ ]:
## Defining the function to calculate the Z - Score
def Z_score(data):
    global outliers,zscore
    outliers = []
    zscore = []
    upper_threshold = 3
    lower_threshold = -3
    mean = np.mean(data)
    std = np.std(data)
    for i in data:
        z_score= (i - mean)/std 
        zscore.append(z_score)
        if np.abs(z_score) > upper_threshold or np.abs(z_score) < lower_threshold:
            outliers.append(i)
    return print("Total number of outliers are",len(outliers)) #outliers,,  zscore

In [ ]:
## Function calling
for column in data_clean.columns:
    Z_score(data_clean[column])
    

Данный метод Z - Score НЕ нашел возможных для удаления строк!

Проведем  графический анализ имеющихся  выбросов данных в data_clean методом sns.boxplot

In [ ]:
drawing_graphs(data_clean, 2, 12, 2, 'boxplot')

У нас в 2-х столбцах наблюаются выбросы после замены 'подозрительных значений' СПРАВА на средние значения +/- 5 значений в столбце и очистки стандартными методами IQR и Z-C.
Проведем дополнительную очистку  в столбцах: 'elasticity_module_stretching' и 'strapery_strength'.
Используем функцию "Методом подбора коэффициентов в df_min_max" и удалением этих выбросов в df.

Проведем иттерацию по удалению строк из нашего df со значениями в df_min_max  при max- строка '1'.

Проведем очистку каждого парметра от значений  'max' лежащих дальше чем 3 СИГМА  'elasticity_module_stretching' и 'strapery_strength'.

In [ ]:
# Функция очистки значений в стлбцах СПРАВА
# пердаваемые параметры:
# df - наш массив
# df_min_max - массив экпертных крайних сначений для каждого столбца
# df_name_column - массив наименований столбцов ,в которых необходимо провести очистку СПРАВА

def clean_column_rigth(df, df_min_max, df_name_column):
    sum_del_rigth = 0 # Количество строк удаленных выбросов СПРАВА после определения столбцов с выбросами в boxplot
    # df_name_column = pd.DataFrame(columns=list_name_column)
    for name_col in df_name_column.columns:
        for column in df.columns:
            if column != name_col:
                continue
            l_begin = len(df)
            df = df[(df[column] < df_min_max.loc[1, column])]     
            l_end = len(df)
            list_x = l_begin - l_end
            print(df.shape)
            print('Количество подозрительных строк в столбце = ', list_x)
            print('---------------------------------')
        sum_del_rigth = sum_del_rigth + list_x
    print('Общее количество удалнных подозрительных строк из-за выбросов справа в df = ', sum_del_rigth)
    #data_clean = data_clean_def  
    return(df)

In [ ]:
# Создадим список столбцов и df, где есть выбросы СПРАВА для передачи в  функцию clean_column_rigth
list_name_column = ['elasticity_module_stretching', 'strapery_strength']
df_name_column = pd.DataFrame(columns=list_name_column)

In [ ]:
data_clean = clean_column_rigth(data_clean, df_min_max, df_name_column)
data_clean.shape

Удалили 11 строк с выбросами справа.

In [ ]:
print('Количество строк до очистки df: ', len(data_composite_inspect))
print('Количество строк после очистки: ', len(data_clean))
print('Удалено строк :', (len(data_composite_inspect) - len(data_clean)))

В начальном DF  у нас было 1023 строки, сейчас осталось 951 строк. Всего удалили  72 строк на данной иттерации

In [ ]:
# Опять проверим на выбросы наш df  на примере графиков 'ящиков с усами' для каждого столбца

In [ ]:
drawing_graphs(data_clean, 2, 12, 2, 'boxplot')

В нашем df опять появились выбросы после удаления 11 из-за выбросов СПРАВА по функции clean_column_rigth

In [ ]:
# DataFrame 'df_min_max' который содержит мин и мах значения по каждому столбу в качестве критерия отбрасывания строк 
# min - индекс сроки 0 и max- индекс строки 1 получены оценочным путем при исследовании гистограмм по каждому параметру-столбцу
# Изменим значение в df_min_max в столбце  'elasticity_module' :[50.0, 1560.0] на значения 'elasticity_module' :[50.0, 1550.0]

df_min_max =  df_min_max.replace({'elasticity_module' : { 50.0 : 50.0, 1560.0 : 1550.0}})
df_min_max

In [ ]:
# Следующая иттерация по удалению выбросов СПРАВА 
# Список столбцов, где есть выбросы СПРАВА для передачи в  функцию clean_column_rigth
list_name_column = ['elasticity_module']
df_name_column = pd.DataFrame(columns=list_name_column)

In [ ]:
# Повторно вызываем функцию clean_column_rigth
data_clean = clean_column_rigth(data_clean, df_min_max, df_name_column)
data_clean.shape

В столбце 'elasticity_module' удалили еще 3 строки

In [ ]:
# Проведем еще одну иттерацию проврки df с  помощью boxplot

In [ ]:
drawing_graphs(data_clean, 2, 12, 2, 'boxplot')

# У нас получились "чистые ", без выброов "ящики с усами"!
Перейдем к следующему этапу исследования нашего datafrema

In [ ]:
print('Количество строк до очистки df: ', len(data_composite_inspect))
print('Количество строк после очистки: ', len(data_clean))
print('Удалено строк :', (len(data_composite_inspect) - len(data_clean)))
total_del_percent = ((len(data_composite_inspect) - len(data_clean)) / len(data_composite_inspect)) * 100
print('Удалено : ', total_del_percent, '%')

В начальном DF  у нас было 1023 строки, сейчас осталось 948 строк. Всего удалили  75 строк или 7.33 %.

# Запишим data_clean в новый файл 'data_main.csv' для дальнейшей работы и передачи в pipline и изучением методами sklearn

In [ ]:
# Запишим data_clean в новый файл 'data_main.csv' для дальнейшей работы и передачи в pipline и изучением методами sklearn
# В массиве 'data_main.csv' мы:
# 1. Заменили нулевые значения '0.0' в строке 19 столбцов 'Шаг нашивки' - 'step_strip'
# и 'Плотность нашивки' - 'density_strip'
# 2.Удалили параметр 'Угол нашивки' - 'pattern_angle' как не несущий существенной информации , так как там всего 2 значения угла  и 90 град распределенных по 50% 
# 3. Провели очистку выбросов методом IQR  и удалили еще 76 строк.
# 4. Метод  Z - Score не нашел дополнительные выбросы
# 5. Проведя графический аналих по boxpot,  мы убедились что в ряде столбцов остались выбросы. Чтобы убрать оставшиеся выбросы,
#  мы в ручном режиме в цикле убрали по шаблону df_min_max значния меньше МИН и больше МАХ . Всего еще 21 строку.
# Всего мы удалили 1023 - 941 = 82 строк  или 8 % из исходного файла 
# Считаем, что df очищен от выбросов

data_clean.to_csv('data_main.csv', index=False, float_format="%.3f")

#  Этап 4 Нормализуем и стандартизируем наш dataset, чтобы привести наши данные к близким размерностям

# В нашем DataSet "data_composite_inspect" есть 12 признаков- столбцов
Но все признаки имеют разную размерность.
Нормализуем и стандартизируем  наш массив 


# Этап 4.1 Нормализация по методу MinMaxskaler

In [ ]:
# список наших столбцов
list_data = list(data_clean)

In [ ]:
minmax_scaler = MinMaxScaler() # вызываем метод MinMaxskaler

In [ ]:
#  обучаем MinMaxScaler() -передаем только числовые значения столбцов  ,указанных списком list_data или можно перечислить какие нужны
data_clean_norm = minmax_scaler.fit_transform(np.array(data_clean[list_data])) 
                                                           

In [ ]:
data_clean_norm[:1] # Проверим первую строку

In [ ]:
# создадим новый df с нормализованными данными с названием наших столбцов 
data_clean_norm_df = pd.DataFrame(data = data_clean_norm, columns = list_data)
data_clean_norm_df.head()

In [ ]:
data_clean_norm_df.describe()

In [ ]:
# Запишем нормализованный очищеный от выбросов df
data_clean_norm_df.to_csv('data_main_norm.csv', index=False, float_format="%.3f")

# 4.2 Проведем стандартизацию по методу StandardSckaler

# from sklearn.preprocessing import StandardScaler

In [ ]:
# список наших столбцов
list_data = list(data_clean)

std_skaler = StandardScaler()

In [ ]:
#  обучаем StandartSkaler() - передаем только числовые значения столбцов  ,указанных списком list_data или можно перечислить какие нужны
data_clean_std = std_skaler.fit_transform(np.array(data_clean[list_data])) 
                                                           

In [ ]:
data_clean_std[:1]

In [ ]:
# создадим новый df со стандартизованными данными с названием наших столбцов  (вернем их)
data_clean_std_df = pd.DataFrame(data = data_clean_std, columns = list_data)
data_clean_std_df.head()

In [ ]:
data_clean_std_df.describe()

In [ ]:
# Запишем стандартизованный очищеный от выбросов df
data_clean_std_df.to_csv('data_main_std.csv', index=False, float_format="%.3f")